# 线性回归的简洁实现
:label:`sec_linear_concise`

在过去的几年里，出于对深度学习强烈的兴趣，
许多公司、学者和业余爱好者开发了各种成熟的开源框架。
这些框架可以自动化基于梯度的学习算法中重复性的工作。
在 :numref:`sec_linear_scratch`中，我们只运用了：
（1）通过张量来进行数据存储和线性代数；
（2）通过自动微分来计算梯度。
实际上，由于数据迭代器、损失函数、优化器和神经网络层很常用，
现代深度学习库也为我们实现了这些组件。

本节将介绍如何(**通过使用深度学习框架来简洁地实现**)
 :numref:`sec_linear_scratch`中的(**线性回归模型**)。

## 生成数据集

与 :numref:`sec_linear_scratch`中类似，我们首先[**生成数据集**]。


In [1]:
import mlx.core as mx
import mlx.nn as nn
import mlx.optimizers as optim
import numpy as np
from d2l import mlx as d2l

In [2]:
true_w = mx.array([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

## 读取数据集

我们可以[**调用框架中现有的API来读取数据**]。
我们将`features`和`labels`作为API的参数传递，并通过数据迭代器指定`batch_size`。
此外，布尔值`is_train`表示是否希望数据迭代器对象在每个迭代周期内打乱数据。


In [3]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """构造一个mlx数据迭代器"""
    X = data_arrays[0]
    y = data_arrays[1]
    perm = mx.array(np.random.permutation(data_arrays[1].size))
    for s in range(0, y.size, batch_size):
        ids = perm[s:s + batch_size]
        yield X[ids], y[ids]

In [4]:
batch_size = 10
data_iter = load_array((features, labels), batch_size)

使用`data_iter`的方式与我们在 :numref:`sec_linear_scratch`中使用`data_iter`函数的方式相同。为了验证是否正常工作，让我们读取并打印第一个小批量样本。
与 :numref:`sec_linear_scratch`不同，这里我们使用`iter`构造Python迭代器，并使用`next`从迭代器中获取第一项。


In [5]:
next(iter(data_iter))

(array([[0.804883, 1.05096],
        [1.85252, 0.887032],
        [-0.0169863, 1.30887],
        ...,
        [-0.127192, 2.23166],
        [-1.76157, -1.0245],
        [-1.39641, -0.0919475]], dtype=float32),
 array([[2.23277],
        [4.86631],
        [-0.285178],
        ...,
        [-3.63524],
        [4.15413],
        [1.72358]], dtype=float32))

## 定义模型

当我们在 :numref:`sec_linear_scratch`中实现线性回归时，
我们明确定义了模型参数变量，并编写了计算的代码，这样通过基本的线性代数运算得到输出。
但是，如果模型变得更加复杂，且当我们几乎每天都需要实现模型时，自然会想简化这个过程。
这种情况类似于为自己的博客从零开始编写网页。
做一两次是有益的，但如果每个新博客就需要工程师花一个月的时间重新开始编写网页，那并不高效。

对于标准深度学习模型，我们可以[**使用框架的预定义好的层**]。这使我们只需关注使用哪些层来构造模型，而不必关注层的实现细节。
我们首先定义一个模型变量`net`，它是一个`Sequential`类的实例。
`Sequential`类将多个层串联在一起。
当给定输入数据时，`Sequential`实例将数据传入到第一层，
然后将第一层的输出作为第二层的输入，以此类推。
在下面的例子中，我们的模型只包含一个层，因此实际上不需要`Sequential`。
但是由于以后几乎所有的模型都是多层的，在这里使用`Sequential`会让你熟悉“标准的流水线”。

回顾 :numref:`fig_single_neuron`中的单层网络架构，
这一单层被称为*全连接层*（fully-connected layer），
因为它的每一个输入都通过矩阵-向量乘法得到它的每个输出。


In [6]:
# nn是神经网络的缩写
class Linear_Regression(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = [nn.Linear(2, 1, bias=True)]

    def __call__(self, x):
        for i, l in enumerate(self.layers):
            x = mx.maximum(x, 0) if i > 0 else x
            x = l(x)
        return x

net = Linear_Regression()
mx.eval(net.parameters()) # evaluate all parameters to initialize the model

## (**初始化模型参数**)

在使用`net`之前，我们需要初始化模型参数。
如在线性回归模型中的权重和偏置。
深度学习框架通常有预定义的方法来初始化参数。
在这里，我们指定每个权重参数应该从均值为0、标准差为0.01的正态分布中随机采样，
偏置参数将初始化为零。


In [7]:
params = net.parameters()
weight_fn = nn.init.normal(mean=0.0, std=0.01)
bias_fn = nn.init.constant(0.0)
net.layers[0].weight = weight_fn(params["layers"][0]["weight"])
net.layers[0].bias = bias_fn(params["layers"][0]["bias"])

## 定义损失函数


In [8]:
mse_loss = nn.losses.mse_loss

## 定义优化算法


In [9]:
optimizer = optim.SGD(learning_rate=0.03)

## 训练

通过深度学习框架的高级API来实现我们的模型只需要相对较少的代码。
我们不必单独分配参数、不必定义我们的损失函数，也不必手动实现小批量随机梯度下降。
当我们需要更复杂的模型时，高级API的优势将大大增加。
当我们有了所有的基本组件，[**训练过程代码与我们从零开始实现时所做的非常相似**]。

回顾一下：在每个迭代周期里，我们将完整遍历一次数据集（`train_data`），
不停地从中获取一个小批量的输入和相应的标签。
对于每一个小批量，我们会进行以下步骤:

* 通过调用`net(X)`生成预测并计算损失`l`（前向传播）。
* 通过进行反向传播来计算梯度。
* 通过调用优化器来更新模型参数。

为了更好的衡量训练效果，我们计算每个迭代周期后的损失，并打印它来监控训练过程。


In [10]:
num_epochs = 3
def loss_fn(model, X, y):
    y_hat = model(X)
    return mse_loss(y_hat, y)

for epoch in range(num_epochs):
    # Generators in Python are one-time iterable. Once a generator has been exhausted, it cannot be restarted or reused.
    for X, y in load_array((features, labels), batch_size):
        loss_and_grad_fn = nn.value_and_grad(net, loss_fn)
        loss, grads = loss_and_grad_fn(net, X, y)
        optimizer.update(net, grads)
        mx.eval(net.state)

    loss = loss_fn(net, features, labels)
    print(f'epoch {epoch + 1}, loss {loss.item():.5f}')

epoch 1, loss 0.00016
epoch 2, loss 0.00010
epoch 3, loss 0.00010


下面我们[**比较生成数据集的真实参数和通过有限数据训练获得的模型参数**]。
要访问参数，我们首先从`net`访问所需的层，然后读取该层的权重和偏置。
正如在从零开始实现中一样，我们估计得到的参数与生成数据的真实参数非常接近。


In [11]:
w = net.layers[0].weight
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net.layers[0].bias
print('b的估计误差：', true_b - b)

w的估计误差： array([-0.000315666, 0.000326872], dtype=float32)
b的估计误差： array([0.000761986], dtype=float32)


## 小结


## 练习

1. 如果将小批量的总损失替换为小批量损失的平均值，需要如何更改学习率？
1. 查看深度学习框架文档，它们提供了哪些损失函数和初始化方法？用Huber损失代替原损失，即
    $$l(y,y') = \begin{cases}|y-y'| -\frac{\sigma}{2} & \text{ if } |y-y'| > \sigma \\ \frac{1}{2 \sigma} (y-y')^2 & \text{ 其它情况}\end{cases}$$
1. 如何访问线性回归的梯度？
